In [ ]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

Machine learning for quantum many-body problems
===============================================

::: {.meta}
:property=\"og:description\": Machine learning for many-body problems
:property=\"og:image\":
<https://pennylane.ai/qml/_images/ml_classical_shadow.png>
:::

::: {.related}
tutorial\_classical\_shadows Classical Shadows
tutorial\_kernel\_based\_training Kernel-based training with
scikit-learn tutorial\_kernels\_module Training and evaluating quantum
kernels
:::

*Author: Utkarsh Azad --- Posted: 02 May 2022. Last Updated: 09 May
2022*

Storing and processing a complete description of an $n$-qubit quantum
mechanical system is challenging because the amount of memory required
generally scales exponentially with the number of qubits. The quantum
community has recently addressed this challenge by using the
`classical shadow <tutorial_classical_shadows>`{.interpreted-text
role="doc"} formalism, which allows us to build more concise classical
descriptions of quantum states using randomized single-qubit
measurements. It was argued in Ref. that combining classical shadows
with classical machine learning enables using learning models that
efficiently predict properties of the quantum systems, such as the
expectation value of a Hamiltonian, correlation functions, and
entanglement entropies.

![Combining machine learning and classical
shadows](/demonstrations/ml_classical_shadows/class_shadow_ml.png){.align-center
width="80.0%"}

In this demo, we describe one of the ideas presented in Ref. for using
classical shadow formalism and machine learning to predict the
ground-state properties of the 2D antiferromagnetic Heisenberg model. We
begin by learning how to build the Heisenberg model, calculate its
ground-state properties, and compute its classical shadow. Finally, we
demonstrate how to use
`kernel-based learning models <tutorial_kernels_module>`{.interpreted-text
role="doc"} to predict ground-state properties from the learned
classical shadows. So let\'s get started!

Building the 2D Heisenberg Model
--------------------------------

We define a two-dimensional antiferromagnetic [Heisenberg
model](https://en.wikipedia.org/wiki/Quantum_Heisenberg_model) as a
square lattice, where a spin-1/2 particle occupies each site. The
antiferromagnetic nature and the overall physics of this model depend on
the couplings $J_{ij}$ present between the spins, as reflected in the
Hamiltonian associated with the model:

$$H = \sum_{i < j} J_{ij}(X_i X_j + Y_i Y_j + Z_i Z_j) .$$

Here, we consider the family of Hamiltonians where all the couplings
$J_{ij}$ are sampled uniformly from \[0, 2\]. We build a coupling matrix
$J$ by providing the number of rows $N_r$ and columns $N_c$ present in
the square lattice. The dimensions of this matrix are $N_s \times N_s$,
where $N_s = N_r \times N_c$ is the total number of spin particles
present in the model.


For this demo, we study a model with four spins arranged on the nodes of
a square lattice. We require four qubits for simulating this model; one
qubit for each spin. We start by building a coupling matrix `J_mat`
using our previously defined function.


We can now visualize the model instance by representing the coupling
matrix as a `networkx` graph:


We then use the same coupling matrix `J_mat` to obtain the Hamiltonian
$H$ for the model we have instantiated above.


For the Heisenberg model, a property of interest is usually the two-body
correlation function $C_{ij}$, which for a pair of spins $i$ and $j$ is
defined as the following operator:

$$\hat{C}_{ij} = \frac{1}{3} (X_i X_j + Y_iY_j + Z_iZ_j).$$


The expectation value of each such operator $\hat{C}_{ij}$ with respect
to the ground state $|\psi_{0}\rangle$ of the model can be used to build
the correlation matrix $C$:

$${C}_{ij} = \langle \hat{C}_{ij} \rangle = \frac{1}{3} \langle \psi_{0} | X_i X_j + Y_iY_j + Z_iZ_j | \psi_{0} \rangle .$$


Hence, to build $C$ for the model, we need to calculate its ground state
$|\psi_{0}\rangle$. We do this by diagonalizing the Hamiltonian for the
model. Then, we obtain the eigenvector corresponding to the smallest
eigenvalue.


We then build a circuit that initializes the qubits into the ground
state and measures the expectation value of the provided set of
observables.


Finally, we execute this circuit to obtain the exact correlation matrix
$C$. We compute the correlation operators $\hat{C}_{ij}$ and their
expectation values with respect to the ground state $|\psi_0\rangle$.


Once built, we can visualize the correlation matrix:


Constructing Classical Shadows
==============================


Now that we have built the Heisenberg model, the next step is to
construct a
`classical shadow <tutorial_classical_shadows>`{.interpreted-text
role="doc"} representation for its ground state. To construct an
approximate classical representation of an $n$-qubit quantum state
$\rho$, we perform randomized single-qubit measurements on $T$-copies of
$\rho$. Each measurement is chosen randomly among the Pauli bases $X$,
$Y$, or $Z$ to yield random $n$ pure product states $|s_i\rangle$ for
each copy:

$$|s_{i}^{(t)}\rangle \in \{|0\rangle, |1\rangle, |+\rangle, |-\rangle, |i+\rangle, |i-\rangle\}.$$

$$S_T(\rho) = \big\{|s_{i}^{(t)}\rangle: i\in\{1,\ldots, n\},\ t\in\{1,\ldots, T\} \big\}.$$

Each of the $|s_i^{(t)}\rangle$ provides us with a snapshot of the state
$\rho$, and the $nT$ measurements yield the complete set $S_{T}$, which
requires just $3nT$ bits to be stored in classical memory. This is
discussed in further detail in our previous demo about
`classical shadows <tutorial_classical_shadows>`{.interpreted-text
role="doc"}.


![](/demonstrations/ml_classical_shadows/class_shadow_prep.png){.align-center
width="100.0%"}


To prepare a classical shadow for the ground state of the Heisenberg
model, we simply reuse the circuit template used above and reconstruct a
`QNode` utilizing a device that performs single-shot measurements.


Now, we define a function to build the classical shadow for the quantum
state prepared by a given $n$-qubit circuit using $T$-copies of
randomized Pauli basis measurements


Furthermore, $S_{T}$ can be used to construct an approximation of the
underlying $n$-qubit state $\rho$ by averaging over $\sigma_t$:

$$\sigma_T(\rho) = \frac{1}{T} \sum_{1}^{T} \big(3|s_{1}^{(t)}\rangle\langle s_1^{(t)}| - \mathbb{I}\big)\otimes \ldots \otimes \big(3|s_{n}^{(t)}\rangle\langle s_n^{(t)}| - \mathbb{I}\big).$$


To see how well the reconstruction works for different values of $T$, we
look at the
[fidelity](https://en.wikipedia.org/wiki/Fidelity_of_quantum_states) of
the actual quantum state with respect to the reconstructed quantum state
from the classical shadow with $T$ copies. On average, as the number of
copies $T$ is increased, the reconstruction becomes more effective with
average higher fidelity values (orange) and lower variance (blue).
Eventually, in the limit $T\rightarrow\infty$, the reconstruction will
be exact.

![Fidelity of the reconstructed ground state with different shadow sizes
$T$](/demonstrations/ml_classical_shadows/fidel_snapshot.png){.align-center
width="80.0%"}


The reconstructed quantum state $\sigma_T$ can also be used to evaluate
expectation values $\text{Tr}(O\sigma_T)$ for some localized observable
$O = \bigotimes_{i}^{n} P_i$, where $P_i \in \{I, X, Y, Z\}$. However,
as shown above, $\sigma_T$ would be only an approximation of $\rho$ for
finite values of $T$. Therefore, to estimate $\langle O \rangle$
robustly, we use the median of means estimation. For this purpose, we
split up the $T$ shadows into $K$ equally-sized groups and evaluate the
median of the mean value of $\langle O \rangle$ for each of these
groups.


Now we estimate the correlation matrix $C^{\prime}$ from the classical
shadow approximation of the ground state.


This time, let us visualize the deviation observed between the exact
correlation matrix ($C$) and the estimated correlation matrix
($C^{\prime}$) to assess the effectiveness of classical shadow
formalism.


Training Classical Machine Learning Models
==========================================


There are multiple ways in which we can combine classical shadows and
machine learning. This could include training a model to learn the
classical representation of quantum systems based on some system
parameter, estimating a property from such learned classical
representations, or a combination of both. In our case, we consider the
problem of using
`kernel-based models <tutorial_kernel_based_training>`{.interpreted-text
role="doc"} to learn the ground-state representation of the Heisenberg
model Hamiltonian $H(x_l)$ from the coupling vector $x_l$, where
$x_l = [J_{i,j} \text{ for } i < j]$. The goal is to predict the
correlation functions $C_{ij}$:

$$\big\{x_l \rightarrow \sigma_T(\rho(x_l)) \rightarrow \text{Tr}(\hat{C}_{ij} \sigma_T(\rho(x_l))) \big\}_{l=1}^{N}.$$

Here, we consider the following kernel-based machine learning model:

$$\hat{\sigma}_{N} (x) = \sum_{l=1}^{N} \kappa(x, x_l)\sigma_T (x_l) = \sum_{l=1}^{N} \left(\sum_{l^{\prime}=1}^{N} k(x, x_{l^{\prime}})(K+\lambda I)^{-1}_{l, l^{\prime}} \sigma_T(x_l) \right),$$

where $\lambda > 0$ is a regularization parameter in cases when $K$ is
not invertible, $\sigma_T(x_l)$ denotes the classical representation of
the ground state $\rho(x_l)$ of the Heisenberg model constructed using
$T$ randomized Pauli measurements, and $K_{ij}=k(x_i, x_j)$ is the
kernel matrix with $k(x, x^{\prime})$ as the kernel function.

Similarly, estimating an expectation value on the predicted ground state
$\sigma_T(x_l)$ using the trained model can then be done by evaluating:

$$\text{Tr}(\hat{O} \hat{\sigma}_{N} (x)) = \sum_{l=1}^{N} \kappa(x, x_l)\text{Tr}(O\sigma_T (x_l)).$$

We train the classical kernel-based models using $N = 70$ randomly
chosen values of the coupling matrices $J$.


In [ ]:
# imports for ML methods and techniques

First, to build the dataset, we use the function `build_dataset` that
takes as input the size of the dataset (`num_points`), the topology of
the lattice (`Nr` and `Nc`), and the number of randomized Pauli
measurements ($T$) for the construction of classical shadows. The
`X_data` is the set of coupling vectors that are defined as a stripped
version of the coupling matrix $J$, where only non-duplicate and
non-zero $J_{ij}$ are considered. The `y_exact` and `y_clean` are the
set of correlation vectors, i.e., the flattened correlation matrix $C$,
computed with respect to the ground-state obtained from exact
diagonalization and classical shadow representation (with $T=500$),
respectively.


Now that our dataset is ready, we can shift our focus to the ML models.
Here, we use two different Kernel functions: (i) Gaussian Kernel and
(ii) Neural Tangent Kernel. For both of them, we consider the
regularization parameter $\lambda$ from the following set of values:

$$\lambda = \left\{ 0.0025, 0.0125, 0.025, 0.05, 0.125, 0.25, 0.5, 1.0, 5.0, 10.0 \right\}.$$

Next, we define the kernel functions $k(x, x^{\prime})$ for each of the
mentioned kernels:


$$k(x, x^{\prime}) = e^{-\gamma||x - x^{\prime}||^{2}_{2}}. \tag{Gaussian Kernel}$$

For the Gaussian kernel, the hyperparameter
$\gamma = N^{2}/\sum_{i=1}^{N} \sum_{j=1}^{N} ||x_i-x_j||^{2}_{2} > 0$
is chosen to be the inverse of the average Euclidean distance $x_i$ and
$x_j$. The kernel is implemented using the radial-basis function (rbf)
kernel in the `sklearn` library.


$$k(x, x^{\prime}) = k^{\text{NTK}}(x, x^{\prime}). \tag{Neural Tangent Kernel}$$

The neural tangent kernel $k^{\text{NTK}}$ used here is equivalent to an
infinite-width feed-forward neural network with four hidden layers and
that uses the rectified linear unit (ReLU) as the activation function.
This is implemented using the `neural_tangents` library.


For the above two defined kernel methods, we obtain the best learning
model by performing hyperparameter tuning using cross-validation for the
prediction task of each $C_{ij}$. For this purpose, we implement the
function `fit_predict_data`, which takes input as the correlation
function index `cij`, kernel matrix `kernel`, and internal kernel
mapping `opt` required by the kernel-based regression models from the
`sklearn` library.


We perform the fitting and prediction for each $C_{ij}$ and print the
output in a tabular format.


In [ ]:
# For each C_ij print (best_cv_score, test_score) pair

Overall, we find that the models with the Gaussian kernel performed
better than those with NTK for predicting the expectation value of the
correlation function $C_{ij}$ for the ground state of the Heisenberg
model. However, the best choice of $\lambda$ differed substantially
across the different $C_{ij}$ for both kernels. We present the predicted
correlation matrix $C^{\prime}$ for randomly selected Heisenberg models
from the test set below for comparison against the actual correlation
matrix $C$, which is obtained from the ground state found using exact
diagonalization.


Finally, we also attempt to showcase the effect of the size of training
data $N$ and the number of Pauli measurements $T$. For this, we look at
the average root-mean-square error (RMSE) in prediction for each kernel
over all two-point correlation functions $C_{ij}$. Here, the first plot
looks at the different training sizes $N$ with a fixed number of
randomized Pauli measurements $T=100$. In contrast, the second plot
looks at the different shadow sizes $T$ with a fixed training data size
$N=70$. The performance improvement seems to be saturating after a
sufficient increase in $N$ and $T$ values for all two kernels in both
the cases.


![image](/demonstrations/ml_classical_shadows/rmse_training.png){width="47.0%"}

![image](/demonstrations/ml_classical_shadows/rmse_shadow.png){width="47.0%"}


Conclusion
==========

This demo illustrates how classical machine learning models can benefit
from the classical shadow formalism for learning characteristics and
predicting the behavior of quantum systems. As argued in Ref., this
raises the possibility that models trained on experimental or quantum
data data can effectively address quantum many-body problems that cannot
be solved using classical methods alone.


References {#ml_classical_shadow_references}
==========

About the author
================
